In [1]:
from bs4 import BeautifulSoup
import urllib
from datetime import datetime
from datetime import timedelta
import pandas as pd
import numpy as np
import operator
from collections import Counter
import pickle
import re
import nltk
from sklearn.externals import joblib
import codecs
from sklearn import feature_extraction
import os
from bs4 import BeautifulSoup
from nltk.stem.snowball import SnowballStemmer
from sklearn.feature_extraction.text import TfidfVectorizer


#this is a function that both tokenizes and stems a given sentence/article 
def token_stem(text):
    # tokenize by sentence and word. this way you ensure you get rid of punctuations
    tokens = [word for sent in nltk.sent_tokenize(text) for word in nltk.word_tokenize(sent)]
    tokens_with_letters = []
    # use the regex library to search only for items that contain letters. this will enable you to eliminate punctuation
    for token in tokens:
        if re.search('[a-zA-Z]', token):
            tokens_with_letters.append(token)
    stems = [stemmer.stem(t) for t in tokens_with_letters] #"stems" part
    return stems

#difference between this one and the one above is the "stems" part. this function is a tokenizer
def token(text): 
    #same as above
    tokens = [word.lower() for sent in nltk.sent_tokenize(text) for word in nltk.word_tokenize(sent)]
    tokens_with_letters = []
    # same as above
    for token in tokens:
        if re.search('[a-zA-Z]', token):
            tokens_with_letters.append(token)
    return tokens_with_letters

def flatten(foo):
    for x in foo:
        if hasattr(x, '__iter__') and not isinstance(x, str):
            for y in flatten(x):
                yield y
        else:
            yield x

#function to get rid of paranthesis and trailing white spaces in the keywords             
def rid_paranthesis_trailing_wspace(words):
    for i in range(len(words)): 
        if "(" in words[i]:
            words[i]=re.match("(.*?)\(",words[i]).group(1) 
            words[i]=words[i].rstrip()            
        else: 
            words[i]=words[i].rstrip()
    return words 

#function to get rid of the \ character in the reuters keywords. treat words separated by this character as two words 
def rid_special_char(words):
    for i in range(len(words)):
        if "/" in words[i]:
            words[i]=words[i].split('/')
    return list(flatten(words))

def make_sentence_out_of_list(words_list):
    sentence=' '.join(word for word in words_list)
    return sentence

nyt_unique=pd.read_csv('nyt_uniques_topicsLabeled.csv')
nyt_labeled_only=nyt_unique.loc[nyt_unique['Section'] != 'Other']
nyt_labeled_only.reset_index(inplace=True)
nyt_labeled_only.drop('index',axis=1,inplace=True)
reuters_unique = pickle.load( open( "20180516-20180621_reuters_unique.pkl", "rb" ) )
reuters_unique.reset_index(inplace=True)
reuters_unique.drop('index',axis=1,inplace=True)

#there is one row that doesn't have keywords for NYT. Might as well manually fix that so we don't run into indexing issues: 
nyt_labeled_only.Keywords[92]='nan'
#for reuters, when we read from the pkl, the keywords are already in "list" format. in NYT this is not the case 
#for one reason or the other. this line of code puts the keywords in NYT in a list, split by a comma
nyt_labeled_only['keyword list']=nyt_labeled_only['Keywords'].apply(lambda x: x.split(',') if(np.all(pd.notnull(x))) else x)
#nyt has "null" values, so we manually make sentences out of lists of keywords by putting an "if" statement 
nyt_labeled_only['sentences']=nyt_labeled_only['keyword list'].apply(lambda words_list:' '.join(word for word in words_list) if(np.all(pd.notnull(words_list))) else words_list)
#only reuters needs the functions below. we first get rid of "\"
#we then get rid of paranthesis and whitespace (because whitespace remains after applying the "rid_special_char" function)
reuters_unique.loc[:, 'keywords'] = reuters_unique.keywords.apply(rid_special_char)
reuters_unique.loc[:, 'keywords'] = reuters_unique.keywords.apply(rid_paranthesis_trailing_wspace)
#we then apply the 'make sentences out of list'. we can apply it here directly bc there are no null values. 
reuters_unique['sentences']=reuters_unique['keywords'].apply(make_sentence_out_of_list)

/Users/berkayadlim/anaconda3/lib/python3.6/site-packages/pandas/core/frame.py:3697: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  errors=errors)
/Users/berkayadlim/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:82: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
/Users/berkayadlim/anaconda3/lib/python3.6/site-packages/IPython/core/interactiveshell.py:2963: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  exec(code_obj, self.user_global_ns, self.user_ns)
/Users/berkayadlim/anacond

In [36]:
reuters_unique.loc[:, 'sentences'] = reuters_unique.sentences.str.replace('north korea', 'nkorea', regex=False)
reuters_unique.loc[:, 'sentences'] = reuters_unique.sentences.str.replace('northkorea', 'nkorea', regex=False)
reuters_unique.loc[:, 'sentences'] = reuters_unique.sentences.str.replace('nothkorea', 'nkorea', regex=False)
nyt_labeled_only.loc[:, 'sentences'] = nyt_labeled_only.sentences.str.replace('north korea', 'nkorea', regex=False)
nyt_labeled_only.loc[:, 'sentences'] = nyt_labeled_only.sentences.str.replace('donald trump', 'dtrump', regex=False)
reuters_unique.loc[:, 'sentences'] = reuters_unique.sentences.str.replace('donald trump', 'dtrump', regex=False)

nyt_list=nyt_labeled_only['sentences'].tolist()
reuters_list=reuters_unique['sentences'].tolist()
nyt_list.extend(reuters_list)

In [16]:
from sklearn.feature_extraction.text import TfidfVectorizer
vectorizer = TfidfVectorizer(use_idf=False, norm='l2',tokenizer=token, ngram_range=(1,3))
%time tfidf_matrix = vectorizer.fit_transform(nyt_list)

CPU times: user 26.3 ms, sys: 22.9 ms, total: 49.2 ms
Wall time: 76.2 ms
